In [1]:
from __future__ import division, absolute_import, print_function
%load_ext autoreload
%autoreload 2

import os 
os.chdir(r"D:\data\ortoCC\design-with-alignments\4heptade-7")
    
import hepran
import hepran.bzipscore as bz
import hepran.bcipa as bc
import hepran.utils as u
from hepran.registers import *
import hepran.agadir as ag

import pandas as pd
import numpy as np
import seaborn as sns
    
%matplotlib inline

import matplotlib as mpl
import matplotlib.pylab as plt

import fcs_utils
fcs_utils.set_plot_style()
from glob import glob    
CCO = os.environ.get("CCO",r"C:\Projekti_KI\ortoCC\CoiledCoilOrtho")

In [2]:
sets = glob("!OUT_bcf/*.set")  
sets += glob("!OUT_additional/*.set")  
sets;

In [4]:
all_fasta = u.load_fasta('all-8k.fasta')

pairs = u.load_set_file('all-8k.pairs')
pairs;

In [15]:
all_on_target = set()
for s in sets:
    all_on_target = all_on_target | bz.pairlist_to_set(u.load_set_file(s))

pairs_df = pd.read_excel("!OUT_additional_pairs/reliable-seq+Tm.xlsx", sheetname="Tm")
pairs_exp = [(str(row.peptide1), str(row.peptide2)) for i, row in pairs_df.iterrows()]
all_on_target =  all_on_target | set(pairs_exp)
all_on_target

{('4H1046-bA', '4H3961-bA'),
 ('4H1046-bH', '4H3961-bH'),
 ('4H1046-bN', '4H3961-bN'),
 ('4H1072-bA', '4H3907-bA'),
 ('4H1072-bH', '4H3907-bH'),
 ('4H1072-bN', '4H3907-bN'),
 ('4H1081-bA', '4H3926-bA'),
 ('4H1081-bH', '4H3926-bH'),
 ('4H1081-bN', '4H3926-bN'),
 ('4H112-bA', '4H2627-bA'),
 ('4H112-bH', '4H2627-bH'),
 ('4H112-bN', '4H2627-bN'),
 ('4H1158-bA', '4H4073-bA'),
 ('4H1158-bH', '4H4073-bH'),
 ('4H1158-bN', '4H4073-bN'),
 ('4H118-bA', '4H2673-bA'),
 ('4H118-bH', '4H2673-bH'),
 ('4H118-bN', '4H2673-bN'),
 ('4H1193-bA', '4H3782-bA'),
 ('4H1193-bA', '4H4038-bA'),
 ('4H1193-bH', '4H3782-bH'),
 ('4H1193-bH', '4H4038-bH'),
 ('4H1193-bN', '4H3782-bN'),
 ('4H1193-bN', '4H4038-bN'),
 ('4H121-bA', '4H2134-bA'),
 ('4H121-bH', '4H2134-bH'),
 ('4H121-bN', '4H2134-bN'),
 ('4H1222-bA', '4H3817-bA'),
 ('4H1222-bH', '4H3817-bH'),
 ('4H1222-bN', '4H3817-bN'),
 ('4H1257-bA', '4H3782-bA'),
 ('4H1257-bH', '4H3782-bH'),
 ('4H1257-bN', '4H3782-bN'),
 ('4H1328-bA', '4H3843-bA'),
 ('4H1328-bH', '4H3843-

In [13]:
SCORES = pd.DataFrame(pairs, columns="ID1 ID2".split())


def get_row_scored(s, fasta, on_target):
    id1 = s['ID1']
    id2 = s['ID2']
    seq1=fasta[id1]
    seq2=fasta[id2]
    
    score_bc0, _, _ = bc.score_aln(seq1, seq2, reg1='f', reg2='f', alignment_type="P00")
    score_bc7, pos_bc7, orient_bc7 = bc.score_aln(seq1, seq2, reg1='f', reg2='f', alignment_type="B07")
    
    score_bz0, _, _ = bz.score_aln(seq1, seq2, reg1='f', reg2='f', alignment_type="P00")
    score_bz7, pos_bz7, orient_bz7 = bz.score_aln(seq1, seq2, reg1='f', reg2='f', alignment_type="B07")
    
    s['score_bc0'] = score_bc0
    s['score_bc7'] = score_bc7
    s['orient_bc7'] = orient_bc7
    s['pos_bc7'] = pos_bc7    
    
    s['score_bz0'] = score_bz0
    s['score_bz7'] = score_bz7
    s['orient_bz7'] = orient_bz7
    s['pos_bz7'] = pos_bz7    
    
    s['on_target'] = (id1, id2) in on_target
    return s
    #print(score_bz7)
    
get_row_scored(SCORES.loc[0], all_fasta, all_on_target)    
SCORES = SCORES.apply(get_row_scored, args=(all_fasta, all_on_target), axis=1)

In [16]:
SCORES.to_excel('all-8k-scores.xlsx')